<a href="https://colab.research.google.com/github/hamagami/ad2025/blob/main/PaDim_timm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PaDim: Patch Distribution Modeling

In [ ]:
# =====================================================
# セル1: MVTec AD を mydrive 直リンクからダウンロード＆展開（Drive不要）
# - TARGET="all" なら全体 (~5GB)
# - それ以外はクラス名（例: "bottle"）のみをダウンロード
# - 展開先: /content/mvtec_anomaly_detection/<class>/...
# - 互換リンク: /content/mvtec -> /content/mvtec_anomaly_detection
# =====================================================

import os, subprocess, tarfile, zipfile, glob

TARGET = "bottle"  # "all" または 15クラス名のいずれか: bottle,cable,capsule,carpet,grid,hazelnut,leather,metal_nut,pill,screw,tile,toothbrush,transistor,wood,zipper
DEST_PARENT = "/content"
DEST_ROOT   = os.path.join(DEST_PARENT, "mvtec_anomaly_detection")
COMP_LINK   = "/content/mvtec"  # 互換リンク

WHOLE_URL = "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938113-1629952094/mvtec_anomaly_detection.tar.xz"
CLASS_URLS = {
    "bottle":      "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937370-1629951468/bottle.tar.xz",
    "cable":       "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937413-1629951498/cable.tar.xz",
    "capsule":     "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937454-1629951595/capsule.tar.xz",
    "carpet":      "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937484-1629951672/carpet.tar.xz",
    "grid":        "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937487-1629951814/grid.tar.xz",
    "hazelnut":    "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937545-1629951845/hazelnut.tar.xz",
    "leather":     "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937607-1629951964/leather.tar.xz",
    "metal_nut":   "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937637-1629952063/metal_nut.tar.xz",
    "pill":        "https://www.mydrive.ch/shares/43421/11a215a5749fcfb75e331ddd5f8e43ee/download/420938129-1629953099/pill.tar.xz",
    "screw":       "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938130-1629953152/screw.tar.xz",
    "tile":        "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938133-1629953189/tile.tar.xz",
    "toothbrush":  "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938134-1629953256/toothbrush.tar.xz",
    "transistor":  "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938166-1629953277/transistor.tar.xz",
    "wood":        "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938383-1629953354/wood.tar.xz",
    "zipper":      "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938385-1629953449/zipper.tar.xz",
}
IMG_EXTS = ('*.png','*.jpg','*.jpeg','*.bmp','*.tif','*.tiff')

def run(cmd):
    print("[RUN]", cmd)
    rc = subprocess.call(cmd, shell=True)
    if rc != 0:
        raise RuntimeError(f"Command failed (rc={rc}): {cmd}")

def list_images_multi_ext(dirpath: str):
    files=[]
    for ext in IMG_EXTS:
        files += glob.glob(os.path.join(dirpath, '**', ext), recursive=True)
    return sorted(files)

os.makedirs(DEST_ROOT, exist_ok=True)

if TARGET == "all":
    tar_path = os.path.join(DEST_PARENT, "mvtec_ad.tar.xz")
    run(f'wget --progress=dot:mega -O "{tar_path}" "{WHOLE_URL}"')
    run(f'tar -xf "{tar_path}" -C "{DEST_PARENT}"')
else:
    if TARGET not in CLASS_URLS:
        raise ValueError(f"Unknown TARGET='{TARGET}'.")
    tar_path = os.path.join(DEST_PARENT, f"{TARGET}.tar.xz")
    run(f'wget --progress=dot:mega -O "{tar_path}" "{CLASS_URLS[TARGET]}"')
    run(f'tar -xf "{tar_path}" -C "{DEST_ROOT}"')

# 互換リンク（/content/mvtec）が無ければ作成
if not os.path.exists(COMP_LINK):
    try:
        os.symlink(DEST_ROOT, COMP_LINK)
        print("[INFO] Symlink:", COMP_LINK, "->", DEST_ROOT)
    except Exception as e:
        print("[WARN] symlink failed:", e)

# 簡易チェック＆要約
def quick_summary(root_parent="/content"):
    candidates = ["/content/mvtec", "/content/mvtec_anomaly_detection"]
    ok = False
    for base in candidates:
        if not os.path.isdir(base): continue
        classes = [d for d in os.listdir(base) if os.path.isdir(os.path.join(base, d))]
        classes = sorted([c for c in classes if os.path.isdir(os.path.join(base,c,"test")) or os.path.isdir(os.path.join(base,c,"train"))])
        if not classes: continue
        print(f"[READY] Found classes under {base}: {classes[:10]}{' ...' if len(classes)>10 else ''}")
        # 1クラスだけ簡易枚数
        cls0 = classes[0]
        tg = os.path.join(base, cls0, "train", "good")
        ttg= os.path.join(base, cls0, "test",  "good")
        n_tg = len(list_images_multi_ext(tg)) if os.path.isdir(tg) else 0
        n_ttg= len(list_images_multi_ext(ttg)) if os.path.isdir(ttg) else 0
        print(f"[SUMMARY] sample class={cls0}: train/good={n_tg}, test/good={n_ttg}")
        ok = True
    if not ok:
        raise FileNotFoundError("Extraction finished but dataset layout not detected. Please check archive.")
    print("[DONE] MVTec AD is ready under /content/mvtec_anomaly_detection")

quick_summary()


In [2]:
# ============================================================
# セル2: PaDiM (timm/ResNet18, layer2+layer3, D=100)
# - /content/mvtec_anomaly_detection/<class>/... を自動検出
# - 学習: 位置ごとガウス(μ, Σ) 推定（チャネルをランダム100次元に縮約）
# - 推論: Mahalanobis距離のマップ → 上位p%平均で画像スコア
# - 閾値: train/good を分割した val の分位点
# - 可視化: ROC/PR, Top異常, 誤検出/見逃し
# ============================================================

import os, glob, math, random
from typing import List, Tuple, Dict
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# timm が無ければ入れる
try:
    import timm
except Exception:
    import sys, subprocess
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "timm"], check=False)
    import timm

from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from scipy.ndimage import gaussian_filter

# ---------------- Config ----------------
CLASS_NAME       = globals().get("TARGET", "bottle")  # セル1のTARGETに合わせる
ROOT_CANDIDATES  = ["/content/mvtec", "/content/mvtec_anomaly_detection"]

IMG_SIZE         = 256
CROP_SIZE        = 224
MEAN             = [0.485, 0.456, 0.406]
STD              = [0.229, 0.224, 0.225]

BATCH            = 16
VAL_SPLIT        = 0.10
OUT_INDICES      = (2, 3)        # timm ResNet18: layer2, layer3
REDUCED_D        = 100           # PaDiM 次元
TOP_P            = 0.02          # 画像スコア＝上位p%平均
TAU_PERC         = 99.5          # val分布の分位点
BLUR_SIGMA       = 2.0           # ヒートマップ平滑化(0/Noneで無効)
EPS_COV          = 1e-5          # 共分散のダイアゴナル微小正則化
SAVE_DIR         = "/content/out_padim_timm"

SEED             = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs(SAVE_DIR, exist_ok=True)
np.random.seed(SEED); random.seed(SEED); torch.manual_seed(SEED)
torch.set_grad_enabled(False)

# ---------------- Utility ----------------
IMG_EXTS = (".png",".jpg",".jpeg",".bmp",".tif",".tiff")
def list_images_multi_ext(dirp):
    files=[]
    for r,_,fs in os.walk(dirp):
        for f in fs:
            if os.path.splitext(f)[1].lower() in IMG_EXTS:
                files.append(os.path.join(r,f))
    return sorted(files)

def autodetect_root(cls: str):
    hits=[]
    for base in ROOT_CANDIDATES:
        p_tr = os.path.join(base, cls, "train", "good")
        p_te = os.path.join(base, cls, "test")
        if os.path.isdir(p_tr) and os.path.isdir(p_te):
            hits.append(base)
    if not hits:
        for base in ["/content"]:
            for p in glob.glob(os.path.join(base, "**", cls, "train", "good"), recursive=True):
                root = p.split(f"/{cls}/train/")[0]
                if os.path.isdir(os.path.join(root, cls, "test")):
                    hits.append(root)
    if not hits:
        raise FileNotFoundError(f"[ERROR] Dataset for class='{cls}' not found under /content.")
    hits = sorted(set(hits), key=lambda x: len(x))
    return hits[0]

def print_dataset_summary(root, cls):
    tg  = os.path.join(root, cls, "train", "good")
    ttg = os.path.join(root, cls, "test", "good")
    tdir= os.path.join(root, cls, "test")
    def _c(p): return len(list_images_multi_ext(p)) if os.path.isdir(p) else 0
    subs=[]
    if os.path.isdir(tdir):
        subs=[d for d in os.listdir(tdir) if os.path.isdir(os.path.join(tdir,d)) and d not in ["good","ground_truth"]]
    n_an=0
    for d in subs: n_an += _c(os.path.join(tdir,d))
    print("[SUMMARY]", {"root":root, "class":cls, "train/good":_c(tg),
                        "test/good":_c(ttg), "test/anoms":n_an, "anomaly_types":subs})

# ---------------- Datasets ----------------
_tf = transforms.Compose([
    transforms.Resize(IMG_SIZE, interpolation=Image.BILINEAR),
    transforms.CenterCrop(CROP_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD),
])

class MVTecTrain(Dataset):
    def __init__(self, root, cls, val_split=0.1, is_val=False):
        base = os.path.join(root, cls, "train", "good")
        files = list_images_multi_ext(base)
        assert len(files)>0, f"No train images in {base}"
        n_val = max(1, int(len(files)*val_split))
        self.files = files[:n_val] if is_val else files[n_val:]
        if is_val and len(self.files)==0: self.files = files[-n_val:]
    def __len__(self): return len(self.files)
    def __getitem__(self, i):
        img = Image.open(self.files[i]).convert("RGB")
        return _tf(img), 0

class MVTecTest(Dataset):
    def __init__(self, root, cls):
        base = os.path.join(root, cls, "test")
        goods = list_images_multi_ext(os.path.join(base, "good"))
        anoms, types = [], []
        for d in os.listdir(base):
            p = os.path.join(base, d)
            if os.path.isdir(p) and d not in ["good","ground_truth"]:
                fs = list_images_multi_ext(p)
                anoms.extend(fs); types.extend([d]*len(fs))
        self.files  = goods + anoms
        self.labels = [0]*len(goods) + [1]*len(anoms)
        self.types  = ["good"]*len(goods) + types
        assert len(self.files)>0, f"No test images in {base}"
    def __len__(self): return len(self.files)
    def __getitem__(self, i):
        img = Image.open(self.files[i]).convert("RGB")
        x = _tf(img)
        return x, self.labels[i], self.files[i], self.types[i]

In [ ]:
# ---------------- Backbone (timm ResNet18) ----------------
backbone = timm.create_model(
    'resnet18.a1_in1k', pretrained=True, features_only=True, out_indices=OUT_INDICES
).to(device).eval()

def extract_feats(x: torch.Tensor) -> torch.Tensor:
    """
    x: (B,3,H,W) → list[ (B,C_i,H_i,W_i) ]  (layer2, layer3)
    layer3 を layer2 の空間解像度に upsample → チャネル連結 → L2正規化
    返り: (B, C_total, Ht, Wt)  ※ここをベース解像度として PaDiM を構築
    """
    feats = backbone(x)
    f2, f3 = feats[0], feats[1]         # e.g., (B,128,28,28), (B,256,14,14)
    f3u = F.interpolate(f3, size=f2.shape[2:], mode='bilinear', align_corners=False)
    f = torch.cat([f2, f3u], dim=1)     # (B,384,28,28)
    f = F.normalize(f, p=2, dim=1)
    return f

def fmap_to_desc(f: torch.Tensor) -> torch.Tensor:
    # (B,C,H,W) → (B, P=H*W, C)
    B,C,H,W = f.shape
    return f.permute(0,2,3,1).contiguous().view(B, H*W, C)

In [ ]:
# ---------------- Dataset/Loader 準備 ----------------
root = autodetect_root(CLASS_NAME)
print_dataset_summary(root, CLASS_NAME)

ds_tr = MVTecTrain(root, CLASS_NAME, val_split=VAL_SPLIT, is_val=False)
ds_va = MVTecTrain(root, CLASS_NAME, val_split=VAL_SPLIT, is_val=True)
ds_te = MVTecTest (root, CLASS_NAME)

dl_tr = DataLoader(ds_tr, batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True)
dl_va = DataLoader(ds_va, batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True)
dl_te = DataLoader(ds_te, batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True)

print(f"[INFO] train={len(ds_tr)} | val={len(ds_va)} | test={len(ds_te)}")

# ---------------- PaDiM: μ, Σ（→Cholesky） 学習 ----------------
# まず1バッチで形を確定し、チャネル縮約(ランダム100次元)のインデックスを決定
xb0, _ = next(iter(dl_tr))
xb0 = xb0.to(device)
f0 = extract_feats(xb0)                   # (B,Ct,Ht,Wt)
_, Ct, Ht, Wt = f0.shape
P = Ht*Wt
rng = torch.Generator().manual_seed(SEED)
CH_IDX = torch.randperm(Ct, generator=rng)[:REDUCED_D].to(device)   # (D,)
print(f"[PaDiM] base fmap: C={Ct}, HxW={Ht}x{Wt}, reduced D={REDUCED_D}")

# 累積量（float64で精度安定）：sum_x (P,D), sum_xxT (P,D,D), n枚数
sum_x   = torch.zeros(P, REDUCED_D, dtype=torch.float64)
sum_xxT = torch.zeros(P, REDUCED_D, REDUCED_D, dtype=torch.float64)
n_imgs  = 0

for xb, _ in dl_tr:
    xb = xb.to(device)
    f  = extract_feats(xb)                # (B,Ct,Ht,Wt)
    f  = f[:, CH_IDX, :, :]               # (B,D,Ht,Wt)
    desc = fmap_to_desc(f).cpu().double() # (B,P,D)
    B = desc.size(0)
    # sum_x
    sum_x += desc.sum(dim=0)              # (P,D)
    # sum_xxT : 各画像で (P,D,D) の outer を積んでバッチ加算
    #  einsum('bpd,bpe->pde') でもOK
    for b in range(B):
        xb_ = desc[b]                     # (P,D)
        sum_xxT += torch.einsum('pd,pe->pde', xb_, xb_)
    n_imgs += B

mu = (sum_x / max(1, n_imgs)).float()     # (P,D)
Exx = (sum_xxT / max(1, n_imgs)).float()  # (P,D,D)
# 共分散: E[xx^T] - μμ^T
mu_outer = torch.einsum('pd,pe->pde', mu, mu)  # (P,D,D)
cov = Exx - mu_outer                           # (P,D,D)

# 共分散 正則化 + Cholesky（失敗時はジッターを増やす）
I = torch.eye(REDUCED_D).unsqueeze(0).repeat(P,1,1)
cov = cov + EPS_COV * I
L = None
for jitter in [0.0, 1e-6, 1e-5, 1e-4, 1e-3]:
    try:
        L = torch.linalg.cholesky(cov + jitter*I)
        if jitter>0: print(f"[INFO] cholesky jitter used: {jitter}")
        break
    except RuntimeError:
        if jitter == 1e-3:
            raise
        continue

# 推論用に GPU へ
mu_d = mu.to(device)            # (P,D)
L_d  = L.to(device)             # (P,D,D)

# ---------------- スコア計算関数（Mahalanobis） ----------------
@torch.no_grad()
def batch_padim_scores(xb: torch.Tensor) -> Tuple[np.ndarray, np.ndarray]:
    """
    xb: (B,3,H,W)
    返り:
      amap_up: (B, CROP_SIZE, CROP_SIZE) 位置スコア（Mahalanobis^2）
      img_sc : (B,) 上位TOP_P%平均の画像スコア
    """
    f = extract_feats(xb)                # (B,Ct,Ht,Wt)
    f = f[:, CH_IDX, :, :]              # (B,D,Ht,Wt)
    desc = fmap_to_desc(f)              # (B,P,D)
    B = desc.size(0)
    # delta: (B,P,D)
    delta = (desc - mu_d.unsqueeze(0))  # broadcast
    # solve L y = delta^T  → y: (P,D,B)
    Bmat = delta.permute(1,2,0)         # (P,D,B)
    y = torch.linalg.solve_triangular(L_d, Bmat, upper=False)  # (P,D,B)
    d2 = (y**2).sum(dim=1).permute(1,0) # (B,P)
    amap = d2.reshape(B, Ht, Wt).detach().cpu().numpy()

    # （任意）平滑化 → 画像サイズへ拡大
    if BLUR_SIGMA and BLUR_SIGMA>0:
        amap = np.stack([gaussian_filter(a, sigma=BLUR_SIGMA) for a in amap], axis=0)
    amap_t = torch.from_numpy(amap).unsqueeze(1).float()
    amap_up = F.interpolate(amap_t, size=(CROP_SIZE, CROP_SIZE), mode='bilinear', align_corners=False).squeeze(1).numpy()

    # 画像スコア（上位p%平均）
    flat = amap_up.reshape(B, -1)
    k = max(1, int(flat.shape[1] * TOP_P))
    img_sc = np.array([np.mean(np.partition(flat[i], -k)[-k:]) for i in range(B)], dtype=np.float32)
    return amap_up, img_sc

# ---------------- 閾値（val） ----------------
val_scores = []
for xb, _ in dl_va:
    xb = xb.to(device)
    amap_up, sc = batch_padim_scores(xb)
    val_scores.extend(sc.tolist())
val_scores = np.asarray(val_scores, dtype=np.float32)
tau = float(np.percentile(val_scores, TAU_PERC))
print(f"[TAU] from val {TAU_PERC}% → tau={tau:.6f}")


In [ ]:
# ---------------- Test 推論＆評価 ----------------
all_scores, all_labels, all_amaps, all_paths, all_types = [], [], [], [], []
for xb, yb, paths, types in dl_te:
    xb = xb.to(device)
    amap_up, sc = batch_padim_scores(xb)
    all_scores.extend(sc.tolist())
    all_labels.extend(yb.numpy().tolist())
    all_paths.extend(list(paths))
    all_types.extend(list(types))
    all_amaps.extend([a for a in amap_up])

scores_np = np.asarray(all_scores, dtype=np.float32)
labels_np = np.asarray(all_labels, dtype=np.int32)

auroc_img = roc_auc_score(labels_np, scores_np)
prec, rec, _ = precision_recall_curve(labels_np, scores_np)
ap_img = average_precision_score(labels_np, scores_np)
print(f"[IMAGE] AUROC={auroc_img:.4f} | AP={ap_img:.4f}")

# 曲線表示
fpr, tpr, thr = roc_curve(labels_np, scores_np)
plt.figure(figsize=(10,4))
plt.subplot(1,2,1); plt.plot(fpr,tpr,label=f"AUC={auroc_img:.4f}"); plt.plot([0,1],[0,1],'--',alpha=0.5)
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("ROC (PaDiM, timm-resnet18)"); plt.legend()
plt.subplot(1,2,2); plt.plot(rec,prec,label=f"AP={ap_img:.4f}")
plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("PR (PaDiM)"); plt.legend()
plt.tight_layout(); plt.show()

In [ ]:
# ---------------- 可視化 ----------------
def imread_rgb(p):
    img = Image.open(p).convert("RGB")
    return np.array(img)

def overlay_show(img_rgb, amap, title, alpha=0.5):
    a = amap
    if a.max() > 0:
        a = (a - a.min()) / (a.max() - a.min() + 1e-8)
    plt.figure(figsize=(8,3))
    ax1 = plt.subplot(1,2,1); ax1.imshow(img_rgb); ax1.set_title("image"); ax1.axis("off")
    ax2 = plt.subplot(1,2,2); ax2.imshow(img_rgb); ax2.imshow(a, cmap="jet", alpha=alpha); ax2.set_title(title); ax2.axis("off")
    plt.tight_layout(); plt.show()

# Top-K 異常
order = np.argsort(-scores_np)
TOPK  = min(6, len(order))
print(f"[SHOW] Top-{TOPK} anomaly candidates")
for r in range(TOPK):
    i = int(order[r])
    img = imread_rgb(all_paths[i])
    overlay_show(img, all_amaps[i], f"score={scores_np[i]:.6f} | pred={int(scores_np[i]>tau)} | true={labels_np[i]}")

# しきい値で混同行列風の枚数
preds = (scores_np > tau).astype(np.int32)
TP = np.where((preds==1)&(labels_np==1))[0]
FP = np.where((preds==1)&(labels_np==0))[0]
FN = np.where((preds==0)&(labels_np==1))[0]
print(f"[CONF] TP={len(TP)} FP={len(FP)} FN={len(FN)} TN={len(labels_np)-len(TP)-len(FP)-len(FN)}")

# 誤検出/見逃し 例
def show_examples(idxs, title):
    if len(idxs)==0:
        print(f"[SHOW] {title}: none"); return
    print(f"[SHOW] {title} examples")
    for i in idxs[:3]:
        img = imread_rgb(all_paths[i])
        overlay_show(img, all_amaps[i], f"{title} | score={scores_np[i]:.6f}")

show_examples(FP, "False Positive")
show_examples(FN, "False Negative")

# ---------------- 保存 ----------------
np.save(os.path.join(SAVE_DIR, f"scores_{CLASS_NAME}.npy"), scores_np)
np.save(os.path.join(SAVE_DIR, f"labels_{CLASS_NAME}.npy"), labels_np)
print(f"[SAVED] {SAVE_DIR}/scores_{CLASS_NAME}.npy, labels_{CLASS_NAME}.npy")


In [ ]:
# ==== Reset notebook metadata in Colab ====
# 使い方:
#   MODE = "soft"  → widgetsメタだけ除去（最小改変）
#   MODE = "hard"  → kernelspec/language_info 以外のメタを全消し（確実）
#   STRIP_OUTPUTS = True で出力と実行番号も削除（軽量化＆差分きれい）

MODE = "hard"          # "soft" or "hard"
STRIP_OUTPUTS = False  # True で全出力/実行番号を削除

from google.colab import _message
import nbformat as nbf
from pathlib import Path

nb_dict = _message.blocking_request('get_ipynb')['ipynb']
name = nb_dict.get('metadata',{}).get('colab',{}).get('name','notebook.ipynb')

def reset_metadata(nb, mode="hard", strip_outputs=False):
    md = nb.get('metadata', {})
    if mode == "soft":
        # widgetsだけ消す
        md.pop('widgets', None)
    else:  # hard
        # 必要最低限だけ残す
        ks = md.get('kernelspec', {'display_name':'Python 3','language':'python','name':'python3'})
        li = md.get('language_info', {'name':'python','pygments_lexer':'ipython3'})
        nb['metadata'] = {'kernelspec': ks, 'language_info': li}

    for c in nb.get('cells', []):
        # セルのwidgetsメタを除去（soft） or 全メタ初期化（hard）
        if mode == "soft":
            c.get('metadata', {}).pop('widgets', None)
        else:
            c['metadata'] = {}

        # ウィジェット出力を除去
        if c.get('cell_type') == 'code' and 'outputs' in c:
            new_out = []
            for out in c['outputs']:
                data = out.get('data', None)
                if isinstance(data, dict):
                    data.pop('application/vnd.jupyter.widget-view+json', None)
                    data.pop('application/vnd.jupyter.widget-state+json', None)
                    if not data:
                        continue
                if not strip_outputs:
                    new_out.append(out)
            c['outputs'] = [] if strip_outputs else new_out
            if strip_outputs:
                c['execution_count'] = None
    return nb

nb_clean = reset_metadata(nb_dict, MODE, STRIP_OUTPUTS)
out_path = f"/content/{name.rsplit('.ipynb',1)[0]}_reset.ipynb"
nbf.write(nbf.from_dict(nb_clean), out_path)
print("Wrote:", out_path)

# 仕上げ確認（GitHubが落ちる元が残っていないか）
print("Top-level widgets exists?:", 'widgets' in nb_clean.get('metadata',{}))
